In [2]:
import geopandas as gpd
import pandas as pd
import contextily as cx
import matplotlib.pyplot as plt
import rasterio as ras
import numpy as np
import gc

Assumes the results from the image preprocessing file are here.

In [3]:
elev = gpd.read_file("elev.geojson")
counties = gpd.read_file("orCounties.geojson")
known_time = gpd.read_file("landslidesKnownTime.geojson").to_crs("EPSG:4326")

In [20]:
center1 = elev.geometry.union_all().centroid

Ensure we are only working with located landslides in the data

In [5]:
inState = counties.union_all(method="coverage").contains(known_time.geometry)

In [6]:
known_time = known_time[inState]

In [7]:
known_time = known_time[["VOLUME_ft3", "geometry", "YEAR"]]

In [8]:
known_time

,VOLUME_ft3,geometry,YEAR
0,1594310.00,POINT (-123.77821 46.16399),2008.0
1,6344.53,POINT (-123.80584 46.17906),2007.0
2,21911.90,POINT (-122.65778 45.39652),2009.0
3,4731.07,POINT (-122.65831 45.39629),2009.0
4,38562.40,POINT (-122.65885 45.3976),2009.0
...,...,...,...
7227,NaN,POINT (-122.18877 43.45249),1996.0
7228,NaN,POINT (-122.42341 43.41715),1996.0
7229,NaN,POINT (-123.78996 43.81432),1996.0
7230,NaN,POINT (-123.79815 43.81184),1996.0


Generate an approximate distance between counties for use in the GP

In [9]:
clist = []
for county in counties.iterrows():
    clist.append(county[1].geometry.contains(known_time.geometry))

In [10]:
countyCentroids = counties.to_crs(epsg=2991).centroid

In [11]:
def distMatrix(one, other):
    return np.sqrt(np.sum(np.square((one[:, None] - other[None, :])), axis=-1))

In [12]:
lon = countyCentroids.x
lat = countyCentroids.y

In [ ]:
countyLocs = np.array([lon.to_numpy(), lat.to_numpy()]).T
countyLocs2 = (countyLocs - np.min(countyLocs, axis=0)) / (0.75 * np.max(countyLocs)) # Scaled version of the above

In [16]:
countyDists2 = distMatrix(countyLocs2,  countyLocs2)

In [ ]:
np.savetxt("county_dist_scale.dat", countyDists2)
np.savetxt("counties_coords.dat", countyLocs2)

Save a matrix indicating which counties and times correspond to which entries.

In [21]:
countyDesign = np.hstack([x.to_numpy()[:, None] for x in clist]) * 1.0

In [22]:
countyDesign.shape

(7222, 36)

In [23]:
times = pd.get_dummies(known_time["YEAR"]).columns.to_numpy()

In [20]:
year_design = pd.get_dummies(known_time["YEAR"], "y_").to_numpy() * 1.0

In [21]:
time_design = year_design

In [24]:
time_dist = distMatrix(times[:, None], times[:, None])

In [28]:
pd.Series(times).to_csv("Years.csv")

Save landslides stratified by county and year for analysis, along with some extra metadata files not used for this project.

In [26]:
known_time_elev = known_time.to_crs("+proj=cea").sjoin_nearest(elev.to_crs("+proj=cea"), "left").to_crs(known_time.crs)

In [27]:
slides = known_time_elev[["VOLUME_ft3", "YEAR", "elev", "elev_std"]]

In [28]:
countyElevStds = []
for county in counties.iterrows():
    county = county[1].geometry
    correspElevs = elev[county.contains(elev.geometry)]
    countyElevStds.append(np.sqrt(np.mean(np.square(correspElevs["elev_std"].to_numpy()))))

In [29]:
counties["county_elev_std"] = np.array(countyElevStds)

In [ ]:
np.savetxt("time_design.dat", time_design, fmt="%.1f")
np.savetxt("time_dist.dat", time_dist, fmt="%.1f")
counties[["COUNTY_NAME", "COBCODE", "county_elev_std"]].to_csv("counties.csv", index = False)
slides.to_csv("slides.csv", index = False)

In [30]:
rSlides = slides.reset_index(drop=True)
y = np.zeros((countyDesign.shape[1], year_design.shape[1])).astype("int64")
for cidx in range(countyDesign.shape[1]):
    countySlides = rSlides[countyDesign[:, cidx].astype("bool")]
    actualYears = np.argwhere(year_design)[:, 1][countySlides.index.to_numpy()]
    for yidx in actualYears:
        y[cidx, yidx] += 1

In [310]:
np.savetxt("landslides_county_year.dat", y, fmt="%.1f")